In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
INCLUDED_CHANNELS = [
    'EEG FP1',
    'EEG FP2',
    'EEG F3',
    'EEG F4',
    'EEG C3',
    'EEG C4',
    'EEG P3',
    'EEG P4',
    'EEG O1',
    'EEG O2',
    'EEG F7',
    'EEG F8',
    'EEG T3',
    'EEG T4',
    'EEG T5',
    'EEG T6',
    'EEG FZ',
    'EEG CZ',
    'EEG PZ'
]

In [3]:
# load distances between sensors according to 10-20 system
dist_df = pd.read_csv('./distances_3d.csv')

In [4]:
dist_df

,from,to,distance
0,EEG FP1,EEG FP1,0.000000
1,EEG FP1,EEG FP2,0.618000
2,EEG FP1,EEG F3,0.618969
3,EEG FP1,EEG F4,1.030322
4,EEG FP1,EEG C3,1.250226
...,...,...,...
356,EEG PZ,EEG T5,1.081066
357,EEG PZ,EEG T6,1.081066
358,EEG PZ,EEG FZ,1.414200
359,EEG PZ,EEG CZ,0.765363


In [5]:
def get_adjacency_matrix(distance_df, sensor_ids, dist_k=0.9):
    """
    Args:
        distance_df: data frame with three columns: [from, to, distance].
        sensor_ids: list of sensor ids.
        dist_k: threshold for graph sparsity
    Returns:
        adj_mx: adj
    """
    num_sensors = len(sensor_ids)
    dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
    dist_mx[:] = np.inf
    # Builds sensor id to index map.
    sensor_id_to_ind = {}
    for i, sensor_id in enumerate(sensor_ids):
        sensor_id_to_ind[sensor_id] = i

    # Fills cells in the matrix with distances.
    for row in distance_df.values:
        if row[0] not in sensor_id_to_ind or row[1] not in sensor_id_to_ind:
            continue
        dist_mx[sensor_id_to_ind[row[0]], sensor_id_to_ind[row[1]]] = row[2]

    # Calculates the standard deviation as theta.
    distances = dist_mx[~np.isinf(dist_mx)].flatten()
    std = distances.std()

    # Sets entries that lower than a threshold, i.e., k, to zero for sparsity.    
    adj_mx = np.exp(-np.square(dist_mx / std))
    adj_mx[dist_mx > dist_k] = 0
   
    return adj_mx, sensor_id_to_ind

In [6]:
thresh = 0.9
adj_mat, sensor_id_to_ind = get_adjacency_matrix(dist_df, INCLUDED_CHANNELS, dist_k=thresh)

In [7]:
sensor_id_to_ind

{'EEG FP1': 0,
 'EEG FP2': 1,
 'EEG F3': 2,
 'EEG F4': 3,
 'EEG C3': 4,
 'EEG C4': 5,
 'EEG P3': 6,
 'EEG P4': 7,
 'EEG O1': 8,
 'EEG O2': 9,
 'EEG F7': 10,
 'EEG F8': 11,
 'EEG T3': 12,
 'EEG T4': 13,
 'EEG T5': 14,
 'EEG T6': 15,
 'EEG FZ': 16,
 'EEG CZ': 17,
 'EEG PZ': 18}

In [8]:
adj_mat.shape

(19, 19)